# Range test analysis

Analysis of acoustic tracking performance using range test data obtained in Mahone Bay near Halifax, NS, by OTN field experiments during March-April 2016.

Components of this notebook:

 * load detection events and deployment metadata about receivers and transmitters
 * analyse detections for each receiver/transmitter combination separately
 * calculate interval lengths between detections and detection rates within fixed time windows
 * calculate further metadata descriptions, e.g. distance between stations, R/T names involving distinctions such as near/far distance or high/low power
 * process tidal data for the time period considering high/low tide times and the observed heights, calculate tidal phase and height change for each detection timestamp via cosine interpolation of heights
 * obtain further environmental variables from external data sources
 * produce different visualizations to study correlation of env. variables and operating conditions with detection rate

**Data:** For most sources for environmental variables data has been collected for 3 hour intervals.

See [installation instructions](../docs/install.md) on how to run this notebook.

In [ ]:
import range_driver as rd
import kadlu
import pandas as pd
%matplotlib inline
rd.mpl_set_notebook_params()

## Obtain configuration from YAML files

In [ ]:
baseconfig = rd.yload(rd.load_file("configs/base_config.yaml"))
viewconfig = rd.yload(rd.load_file("configs/view_config.yaml"))
rd.deep_update(baseconfig, viewconfig)

rawconfig = rd.yload(rd.load_file("configs/mahone_config.yaml"))
config = rd.merge_dicts(baseconfig, rawconfig)

rd.prepare_config(config)

# Set the kadlu storage location to point to cached data
kadlu.storage_cfg(setdir=f'{rd.utils.repo_path()}/data/MahoneBay2016/kadlu_data/')

## Read Detections, Metadata, and Environmental Variables

In [ ]:
dets = rd.Detections(config)

## Detail Reports

In [ ]:
rd.report_group_info(dets)
rd.report_tidal(dets)

### Data Sources

In [ ]:
print(rd.kadlu_source_map())
print("Configured data sources:\n")
print(rd.ydump(config.data.sources))

## Maps visualization
Map view including region of interest bound, environmental data source nodes, receiver and tag locations.

In [ ]:
rd.report_map_view(dets)

In [ ]:
# --------------------- Detail plots ------------------------
columns, colnames, column = rd.get_column_info(config)
rd.displaymd("## Plots of detection density and interval lengths <br/> against tidal phase (t2) and {}".format(colnames[column]))

In [ ]:
rd.report_all_group_plots(dets)

In [ ]:
rd.report_heatmap(dets)

# Discussion

In the above visual summary, the **H-N** combinations, i.e. high-power, near distance, are the ones where water velocity shows the least effect on variations in detection rate. This confirms expectations and shows promise for the proposed study method. Next steps include:

- Continue to work with detection rate (DR) as calculated in a fixed grid of time windows
- Compare variations of DR with respect to other environmental variables
- Import other environmental variables automatically via data source APIs (ERDDAP, kadlu.fetch)
- Compare different scenarios of factor importance depending on environmental setting of the study 


# Acknowledgements

The above analysis was performed using [data from OTN](http://members.devel.oceantrack.org/erddap/tabledap/otnunit_aat_detections.html) (provided by Jonathan Pye of OTN), in combination with HYCOM environmental data and tidal data provided by Casey Hilliard (Meridian/Dal), with a synthesized dataset prepared by Matthew Berkowitz (SFU), with project definition and guidance provided by Oliver Kirsebom (Dal) and Ines Hessler (Dal) as part of the [Meridian Network](https://meridian.cs.dal.ca).